In [4]:
library("C50")
library("caret")

Loading required package: lattice
Loading required package: ggplot2


1 Import the data and shuffle and split it into a training set (20,000 records), a validation set (8,000 records), and a test set (remaining records).

In [5]:
data <- read.csv('Loan_processed.csv')

# shuffle 
data <- data[sample(nrow(data)),]

# split
trainSplit <- sample(nrow(data), 20000)
train <- data[trainSplit, ]

rest <- data[-trainSplit, ]

validationSplit <- sample(nrow(rest), 8000)
validation <- rest[validationSplit, ]
test <- rest[-validationSplit, ]

2 Using a classification tree, try to predict with an accuracy greater than #repaid loans / (#repaid loans + #charged off loans) if a loan will be repaid.

In [6]:
target <- nrow(data[data$loan_status == "Fully Paid", ]) / 
  (nrow(data[data$loan_status == "Fully Paid", ]) + nrow(data[data$loan_status == "Charged Off", ]))

target

[1] 0.8595498

In [10]:
# fit tree model
treeModel <- C5.0(x = train[, -8], y = train$loan_status)

# make predictions on validation data
predValidation <- predict(treeModel, validation[, -8])

# confusion matrix for results on valiation set
mTtrain <- confusionMatrix(predValidation, validation[, 8], positive = "Charged Off", 
                dnn = c("Prediction", "Reference"))


# make prediction on training data
predTraining <- predict(treeModel, train[, -8])

# confusion matrix for results on training set
mTval <- confusionMatrix(predTraining, train[, 8], positive = "Charged Off", 
                dnn = c("Prediction", "Reference"))

# different way to calculate accuracy
# sum(predValidation == validation[, 8]) / length(predValidation)

The accuracy on the training set is: 0.8584
The accuracy on the validation set is: 0.8575
The target accuracy is : 0.8595498

3 Experiment with the cost matrix. To achieve sensitivities of 25%, 40%, 50%.

In order to achieve this we have to increase to cost of predicting a "Charged Off" loans as "Fully Paid". 

Model with ~25% sensitivity.

In [11]:
cost.matrix25 <- matrix(c(0, 0, 2.76, 0), 2, 2, byrow=TRUE)
rownames(cost.matrix25) <- colnames(cost.matrix25) <- c("Charged Off",
                                                    "Fully Paid")

# model with cost matrix
treeModelCM25 <- C5.0(x = train[, -8], y = train$loan_status, costs = cost.matrix25)

# use model on validation set
predValidationCost25 <- predict(treeModelCM25, validation[, -8])

# confusion matrix for results on training set
m25 <- confusionMatrix(predValidationCost25, validation[, 8], positive = "Charged Off", dnn = c("Prediction", "Reference"))
m25$byClass[1]
m25$byClass[5]
m25$table

Sensitivity 
  0.2187785

Precision 
 0.268757

             Reference
Prediction    Charged Off Fully Paid
  Charged Off         240        653
  Fully Paid          857       6250

Model with ~40% sensitivity.

In [12]:
cost.matrix40 <- matrix(c(0, 0, 3.73, 0), 2, 2, byrow=TRUE)
rownames(cost.matrix40) <- colnames(cost.matrix40) <- c("Charged Off",
                                                    "Fully Paid")

# model with cost matrix
treeModelCM40 <- C5.0(x = train[, -8], y = train$loan_status, costs = cost.matrix40)

# use model on validation set
predValidationCost40 <- predict(treeModelCM40, validation[, -8])

# confusion matrix for results on training set
m40 <- confusionMatrix(predValidationCost40, validation[, 8], positive = "Charged Off", dnn = c("Prediction", "Reference"))
m40$byClass[1]
m40$byClass[5]
m40$table

Sensitivity 
  0.4111212

Precision 
0.2347736

             Reference
Prediction    Charged Off Fully Paid
  Charged Off         451       1470
  Fully Paid          646       5433

Model with ~50% sensitivity.

In [13]:
# cost matrix ~50%
cost.matrix50 <- matrix(c(0, 0, 4.7, 0), 2, 2, byrow=TRUE)
rownames(cost.matrix50) <- colnames(cost.matrix50) <- c("Charged Off",
                                                    "Fully Paid")

# model with cost matrix
treeModelCM50 <- C5.0(x = train[, -8], y = train$loan_status, costs = cost.matrix50)

# use model on validation set
predValidationCost50 <- predict(treeModelCM50, validation[, -8])

# confusion matrix for results on training set
m50 <- confusionMatrix(predValidationCost50, validation[, 8], positive = "Charged Off", dnn = c("Prediction", "Reference"))
m50$byClass[1]
m50$byClass[5]
m50$table

Sensitivity 
  0.4667274

Precision 
0.2120961

             Reference
Prediction    Charged Off Fully Paid
  Charged Off         512       1902
  Fully Paid          585       5001

4 Pick a cost parameter matrix that you assess as the most appropriate for identifying loan applications that deserve further examination.